In [1]:
import pandas as pd
import xlrd
import os
from collections import defaultdict

In [2]:
#from PyPDF2 import PdfFileReader,PdfFileWriter

In [6]:
ls ./data

2019_12/ 2020_01/ 2020_03/ 2020_04/


In [19]:
class Bill:
    def __init__(self, year, mon):
        self.year = year
        self.mon = mon
        self.sell_file = self.get_sell_file()
        content = xlrd.open_workbook(filename=self.sell_file,encoding_override='gbk')
        self.sell_df = pd.read_excel(content)
        self.commission_file = self.get_commission_file()
        content = xlrd.open_workbook(filename=self.commission_file,encoding_override='gbk')
        self.commission_df = pd.read_excel(content)
        #self.sell_df = pd.read_excel(self.sell_file,encoding='gbk')
    def gen_path(self, filename):
        return  './data/{0}_{1:02d}/{2}'.format(self.year, self.mon,filename)
    def get_sell_file(self):
        return self.gen_path('销售单据表.xls')
    def get_commission_file(self):
        return self.gen_path('提成明细表.xls')
    def get_total_income(self):
        #总收入
        self.total_income = 0
        #充值总收入
        self.total_recharge = 0
        #充次总收入
        self.total_time_recharge = 0
        for index, row in self.sell_df.iterrows():
            income_sf = row['实付']
            income_czk = row['储值卡']
            self.total_income += row['实付'] - row['储值卡']
            if row['单据类型'] == '购买次数':
                self.total_time_recharge += row['实付']
            if row['单据类型'] == '购买次数' or row['单据类型'] == '充值收费' or row['单据类型']=='开户收费':
                self.total_recharge += row['实付'] - row['储值卡']
        print('总收入:',self.total_income)
        print('充值总收入:',self.total_recharge)
        print('充次总收入:',self.total_time_recharge)
    def get_true_income(self):
        # 实际项目总收入（会员+非会员）
        self.true_income = 0
        true_income_types = ['消费收银','快速消费']
        #会员实际项目总收入
        self.member_true_income = 0
        for index, row in self.sell_df.iterrows():
            if row['单据类型'] in true_income_types:
                if not pd.isnull(row['会员类型']):
                    self.member_true_income += row['实付']
                self.true_income += row['实付']
        print(self.member_true_income)
        print(self.true_income)
        
        #实际扣次项目
        self.true_time_dic = defaultdict(int)
        for index, row in self.commission_df.iterrows():
            if row['提成类型'] == '扣次消费':
                self.true_time_dic[row['名称']] += 1
        print(self.true_time_dic)
    def get_employee_commission(self):
        tmp_commission = self.commission_df.copy()
        selected_df = tmp_commission[['时间','员工','名称','支付','提成','提成类型','姓名']]
        selected_df['时间'] = selected_df['时间'].apply(lambda x:'{0}-{1}'.format(x.month, x.day))
        for employee, df in selected_df.groupby(['员工']):
            print(employee)
            print(df)
            df = df[['时间','员工','名称','提成类型','姓名','支付','提成']]
            print(df['提成'].sum())
            df.to_csv(self.gen_path('{0}.csv'.format(employee)),index=False,encoding='gbk')
            df.to_excel(self.gen_path('{0}.xls'.format(employee)),index=False,encoding='gbk')
    def get_recharge_detail(self):
        tmp_df = self.sell_df.copy()
        recharge_df = tmp_df[tmp_df['单据类型'].isin(set(['购买次数','充值收费','开户收费']))]
        recharge_df = recharge_df[['单据类型','时间','实付','储值卡','会员姓名','会员卡号']]
        total_recharge = recharge_df['实付'].sum()-recharge_df['储值卡'].sum()
        des_sum = '累计充值：{0},各个会员充值情况如下\n'.format(total_recharge)
        des_sum += '==============================\n'
        for name, df in recharge_df.groupby(['会员卡号']):
            s = 0
            des = '{0}({1:d}):累计充值{2}元\n'.format(df.iloc[0]['会员姓名'],int(name),int(df['实付'].sum()-df['储值卡'].sum()))
            des_sum += des
            #print('{0}:{1}'.format(name,df['实付'].sum()-df['储值卡'].sum()))
            total_recharge += df['实付'].sum()-df['储值卡'].sum()
            s += df['实付'].sum()-df['储值卡'].sum()
            des = '明细如下:\n'
            df = df.sort_values(by=['单据类型'],ascending=False)

            for index, row in df.iterrows():
                if row['单据类型'] == '购买次数' and row['实付']>0:
                    des = des + '购买次数:' + str(int((row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '充值收费' and s > 0:
                    des = des + '充值收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '开户收费' and s > 0:
                    des = des + '开户收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
            #print(des)
            des_sum += des
            des_sum += '==============================\n'
            f = open(self.gen_path('会员充值明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
            f.write(des_sum)
            f.close()
        return des_sum
    def get_nomember_detail(self):
        tmp_df = self.sell_df.copy()
        tmp_df['not_member'] = tmp_df['会员卡号'].apply(lambda x:pd.isnull(x))
        nomember_detail_df = tmp_df[tmp_df['not_member']]
        nomember_detail_df = nomember_detail_df[['单据类型','实付','时间']]
        #print(nomember_detail_df)
        des_not_member = ''
        des_not_member += '散客累计消费:{0}\n明细如下：\n'.format(nomember_detail_df['实付'].sum())
        for index, row in nomember_detail_df.iterrows():
            des_not_member += '{0}:'.format(row['单据类型']) + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
        f = open(self.gen_path('散客消费明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
        f.write(des_not_member)
        f.close()
        return des_not_member

In [20]:
ls ./data/2020_04/

孙妮.csv                       销售单据表.xls
黄晴.csv                       会员充值明细_2020_4.txt
邵红梅.csv                     散客消费明细_2020_4.txt
提成明细表.xls


In [21]:
bill = Bill(2020, 4)

In [22]:
bill.get_sell_file()

'./data/2020_04/销售单据表.xls'

In [23]:
bill.get_true_income()

2538
3148
defaultdict(<class 'int'>, {'暖宫艾灸': 5, '精油开背': 3, '皮肤清洁护理': 1})


In [24]:
bill.sell_file

'./data/2020_04/销售单据表.xls'

In [25]:
bill.get_total_income()

总收入: 4163
充值总收入: 3500
充次总收入: 0


In [26]:
bill.commission_file

'./data/2020_04/提成明细表.xls'

In [27]:
bill.get_employee_commission()

孙妮
      时间  员工    名称    支付     提成  提成类型   姓名
0    4-1  孙妮   NaN   500   25.0  充值收费   黄婷
1    4-1  孙妮   NaN   194   77.6  快速消费   黄婷
2    4-1  孙妮   NaN   182   39.5  快速消费  小米粥
3    4-3  孙妮  暖宫艾灸    88    8.0  扣次消费   潘潘
4    4-6  孙妮  暖宫艾灸    88    8.0  扣次消费   潘潘
5   4-10  孙妮   NaN    69   27.6  快速消费   黄婷
6   4-10  孙妮   NaN   194   77.6  快速消费   张依
7   4-10  孙妮   NaN   194   77.6  快速消费   刘梅
8   4-13  孙妮   NaN   194   77.6  快速消费   姣爷
9   4-13  孙妮   NaN   100   40.0  快速消费  NaN
10  4-14  孙妮   NaN    69   27.6  快速消费   刘梅
11  4-14  孙妮   NaN    20    8.0  快速消费  NaN
12  4-14  孙妮  暖宫艾灸    88    8.0  扣次消费   潘潘
13  4-29  孙妮   NaN  1000   50.0  开户收费   王敏
14  4-29  孙妮   NaN    53   21.2  快速消费   王敏
15  4-29  孙妮   NaN  1000   50.0  充值收费  小倪姐
16  4-29  孙妮   NaN   294  117.6  快速消费   小郭
17  4-29  孙妮   NaN  1000   50.0  开户收费  王丹婧
18  4-29  孙妮   NaN   149   59.6  快速消费  王丹婧
19  4-29  孙妮   NaN   230   92.0  快速消费  NaN
20  4-29  孙妮   NaN    80   32.0  快速消费  NaN
21  4-29  孙妮   NaN    80   32.0  快速消费  NaN
22  4-29

/Users/mtdp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
des_sum = bill.get_recharge_detail()

In [29]:
des_sum

'累计充值：3500,各个会员充值情况如下\n==============================\n黄婷(60006):累计充值500元\n明细如下:\n充值收费:500,时间:4-1\n==============================\n小倪姐(60009):累计充值1000元\n明细如下:\n充值收费:1000,时间:4-29\n==============================\n王敏(60069):累计充值1000元\n明细如下:\n开户收费:1000,时间:4-29\n==============================\n王丹婧(60070):累计充值1000元\n明细如下:\n开户收费:1000,时间:4-29\n==============================\n'

In [30]:
des_not_member = bill.get_nomember_detail()

In [31]:
des_not_member

'散客累计消费:610\n明细如下：\n快速消费:100,时间:4-13\n快速消费:20,时间:4-14\n快速消费:230,时间:4-29\n快速消费:80,时间:4-29\n快速消费:80,时间:4-29\n快速消费:100,时间:4-30\n'